In [33]:
import glob
import os
import pickle
import json

import cv2
import numpy as np

In [7]:
%matplotlib inline
from matplotlib import pyplot as plt

In [2]:
DATA_DIR = os.path.join('../data')
VEHICLES = os.path.join(DATA_DIR, 'vehicles')
NON_VEHICLES = os.path.join(DATA_DIR, 'non-vehicles')
DIRS = ["GTI_Far", "GTI_Left", "GTI_MiddleClose", "GTI_Right"]

In [8]:
from skimage.feature import hog

In [68]:
def hog_features(img):
    rhf = hog(img[:,:,0], orientations=9, pixels_per_cell=(8, 8), 
                     cells_per_block=(3, 3), feature_vector=True)
    ghf = hog(img[:,:,1], orientations=9, pixels_per_cell=(8, 8), 
                     cells_per_block=(3, 3), feature_vector=True)
    bhf = hog(img[:,:,2], orientations=9, pixels_per_cell=(8, 8), 
                     cells_per_block=(3, 3), feature_vector=True)
    return np.hstack((rhf, ghf, bhf))

In [69]:
def read(data_dir, extract_feature):
    for img_path in glob.iglob(os.path.join(data_dir, 'GTI*', '*.png')):
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        yield extract_feature(img)

In [70]:
vehicles = [feature for feature in read(VEHICLES, hog_features)]

/Users/Sumukh/Applications/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


In [71]:
non_vehicles = [feature for feature in read(NON_VEHICLES, hog_features)]

/Users/Sumukh/Applications/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


In [72]:
len(vehicles)

2826

In [73]:
len(non_vehicles)

3900

In [46]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [74]:
data = np.vstack((vehicles, non_vehicles))
labels = np.hstack((np.ones(len(vehicles), np.int32), np.zeros(len(non_vehicles), np.int32)))

In [75]:
len(labels) == len(data)

True

In [76]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)

In [87]:
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix

In [88]:
clf = svm.LinearSVC()
scaler = StandardScaler()
pipeline = Pipeline([('scaler', scaler), ('svc', clf)])
pipeline.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svc', LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [89]:
acc = pipeline.score(x_test, y_test)

In [90]:
pred = pipeline.predict(x_test)

In [91]:
cm = confusion_matrix(y_test, pred)

In [85]:
cm

array([[760,  11],
       [ 15, 560]])

In [92]:
cm

array([[756,  15],
       [ 13, 562]])

In [93]:
acc

0.9791976225854383